In [1]:
! pip install pandas
! pip install openpyxl

In [2]:
import pandas as pd

In [3]:
from numpy import *


class MatchRating:
    original_rating_name = "Original"
    avg_player_name = "AVG"
    max_iterations = 1000

    def __init__(self, data_sheet_file, rating_scale_base, rating_scale_divisor, rating_base, tie_in_weight):

        self.rating_scale_base = rating_scale_base
        self.rating_scale_divisor = rating_scale_divisor
        self.ranking_boost = rating_base
        self.tie_in_weight = tie_in_weight
        
        self.players: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Players", index_col="ShortName")
        self.matches: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Matches")
        self.matches["IsTieInMatch"] = False
        
        match_columns = self.matches.columns.to_list()
        self.winner_columns = [column for column in match_columns if column.startswith("Winner")]
        self.loser_columns = [column for column in match_columns if column.startswith("Loser")]

        print('Number of matches loaded:' + str(len(self.matches)))

        # --- Add tie-in matches ---
        cols = self.matches.columns
        first_winner = cols[cols.str.startswith("Winner")].values[0]
        first_loser = cols[cols.str.startswith("Loser")].values[0]
        
        tie_in_rows = []
        for player in self.players.index:
            tie_in_rows.append({first_winner: player, first_loser: MatchRating.avg_player_name, "K": self.tie_in_weight})
            tie_in_rows.append({first_winner: MatchRating.avg_player_name, first_loser: player, "K": self.tie_in_weight})

        tie_in_matches = pd.DataFrame(tie_in_rows)
        tie_in_matches["IsTieInMatch"] = True
        self.matches = pd.concat([self.matches, tie_in_matches], ignore_index=True)

        self.__player_ratings: pd.DataFrame = pd.DataFrame(columns=["Rating"], index=self.players.index)
        self.__player_ratings["Rating"] = 0.0
        
        

    def calculate_ratings(self, step_sizes: list, min_player_match_count: int):

        def apply_ranking_boost(rankings):
            rankings["Rating"] = rankings["Rating"] - rankings["Rating"].loc[MatchRating.avg_player_name] + self.ranking_boost
            return rankings
        
        def remove_excluded_players(rankings):
            excluded_players = self.players[self.players["Excluded"] == True].index
            rankings = rankings.drop(excluded_players, errors="ignore")
            return rankings.drop(columns=["Excluded"])
        
        def add_player_metadata(rankings):
            return rankings.merge(self.players, left_index=True, right_index=True)
        
        def add_match_metadata(rankings):
            
            def get_wins_and_losses(player):
                matches = self.matches[self.matches["IsTieInMatch"] == False]
                wins = matches[self.winner_columns].apply(lambda x: (player in x.values), axis=1).sum()
                losses = matches[self.loser_columns].apply(lambda x: (player in x.values), axis=1).sum()
                return pd.Series([wins, losses], index=["Wins", "Losses"])
            
            rankings[["Wins", "Losses"]] = rankings["ShortName"].apply(get_wins_and_losses)
            return rankings
        
        def filter_min_match_count(rankings):
            return rankings[rankings["Wins"] + rankings["Losses"] >= min_player_match_count]

        def add_opponent_rating(rankings):

            def get_opponent_rating(player):
                matches = self.matches[(self.matches["IsTieInMatch"] == False) & self.matches.apply(lambda x: (player in x.values), axis=1)]
                matches = matches[self.winner_columns + self.loser_columns]

                def get_match_opponent_rating(match):
                    if player in match[self.winner_columns].values:
                        team_columns = self.winner_columns
                        opponent_columns = self.loser_columns
                    else:
                        team_columns = self.loser_columns
                        opponent_columns = self.winner_columns

                    team_players = match[team_columns].dropna().values
                    opponent_players = match[opponent_columns].dropna().values

                    player_team_ranking = rankings.loc[team_players, ["Rating"]].mean().values[0]
                    opponent_team_ranking = rankings.loc[opponent_players, ["Rating"]].mean().values[0]

                    return mean(opponent_team_ranking - player_team_ranking)

                opponent_ratings = matches.apply(get_match_opponent_rating, axis=1)
                return round(opponent_ratings.mean(), 0)
        
            rankings["AVG_Opponent_Rating"] = rankings["ShortName"].apply(get_opponent_rating)
            return rankings
        
        def finalize_rankings(rankings):
            rankings = rankings.sort_values(by="Rating", ascending=False).reset_index(drop=True)
            rankings.index += 1
            rankings["Rating"] = rankings["Rating"].round(0).astype(int)
            return rankings.reset_index()
        

        for step_size in step_sizes:
            rankings = self.__calculate_ratings(step_size)

        rankings["ShortName"] = rankings.index

        
        rankings = apply_ranking_boost(rankings)
        rankings = add_player_metadata(rankings)
        rankings = add_match_metadata(rankings)
        rankings = add_opponent_rating(rankings)
        rankings = remove_excluded_players(rankings)
        rankings = filter_min_match_count(rankings)
        rankings = finalize_rankings(rankings)

        return rankings


        
    
    def __calculate_ratings(self, step_size):
        
        total_match_weights  = self.matches["K"].sum()

        for i in range(MatchRating.max_iterations):
            rating_table = self.__create_rating_table(step_size)
            
            match_score_accuracy = self.__calculate_match_score_accuracy(rating_table)        
            
            player_rating_accuracy = pow(match_score_accuracy.prod(), 1 / total_match_weights)
            accuracy_product = player_rating_accuracy.sort_values(ascending=False)

            best_accuracy = accuracy_product.index[0]
            print(f"Step Size={step_size}, Iteration {i + 1}, Accuracy: {round(accuracy_product.iloc[0], 5)}")

            if best_accuracy == MatchRating.original_rating_name:
                return self.__player_ratings
            
            player, rating_change = best_accuracy            
            self.__player_ratings.loc[player, ["Rating"]] += rating_change
            
        return self.__player_ratings

    def __calculate_match_score_accuracy(self, rating_table: pd.DataFrame):

        match_columns = self.matches.columns.to_list()
        winner_columns = [column for column in match_columns if column.startswith("Winner")]
        loser_columns = [column for column in match_columns if column.startswith("Loser")]

        def apply_match_score_accuracy(match):
            winners = match[winner_columns]
            winners = winners[winners.notna()]
            winner_rating = rating_table.loc[winners].mean()
            
            losers = match[loser_columns]
            losers = losers[losers.notna()]
            loser_rating = rating_table.loc[losers].sum()

            accuracy = pow(winner_rating / (winner_rating + loser_rating), match["K"])

            return accuracy

        return self.matches.apply(apply_match_score_accuracy, axis=1, result_type="expand")

    def __create_rating_table(self, step_size):
        
        ratings = []
        columns = []

        original_rating = self.__player_ratings["Rating"]

        def add_rating(rating, name):
            ratings.append(rating.apply(lambda x: pow(self.rating_scale_base, (x / self.rating_scale_divisor))))
            columns.append(name)

        def add_alternative_rating(player, rating_change):
            alternative_rating = original_rating.copy()
            alternative_rating.loc[player] += rating_change
                
            add_rating(alternative_rating, (player, rating_change))

        add_rating(original_rating, MatchRating.original_rating_name)

        for player in self.players.index:
            add_alternative_rating(player, step_size)
            add_alternative_rating(player, -step_size)
            
        rating_table = pd.concat(ratings, axis=1)
        rating_table.columns = columns

        return rating_table

In [4]:
rating_system = MatchRating(
    # data_sheet_file="Data/Test.xlsx",
    data_sheet_file="Data/UCH_Data.xlsx",
    rating_scale_base=10,
    rating_scale_divisor=400,
    rating_base=1000,
    tie_in_weight=0.5
)

Number of matches loaded:2


In [5]:
rankings = rating_system.calculate_ratings(
    step_sizes=[1024,512,256,128],
    min_player_match_count=0
)
rankings

Step Size=1024, Iteration 1, Accuracy: 0.43584
Step Size=512, Iteration 1, Accuracy: 0.43584
Step Size=256, Iteration 1, Accuracy: 0.43584
Step Size=128, Iteration 1, Accuracy: 0.43584


,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1000,A,AAA,1,1,0.0
1,2,1000,B,BBB,1,1,0.0
2,3,1000,C,CCC,1,1,0.0
3,4,1000,D,DDD,1,1,0.0


In [6]:
rankings = rating_system.calculate_ratings(
    step_sizes=[64,32,16,8],
    min_player_match_count=0
)
rankings

Step Size=64, Iteration 1, Accuracy: 0.43584
Step Size=32, Iteration 1, Accuracy: 0.43595
Step Size=32, Iteration 2, Accuracy: 0.43595
Step Size=16, Iteration 1, Accuracy: 0.43597
Step Size=16, Iteration 2, Accuracy: 0.43605
Step Size=16, Iteration 3, Accuracy: 0.43606
Step Size=16, Iteration 4, Accuracy: 0.43606
Step Size=8, Iteration 1, Accuracy: 0.43607
Step Size=8, Iteration 2, Accuracy: 0.43609
Step Size=8, Iteration 3, Accuracy: 0.4361
Step Size=8, Iteration 4, Accuracy: 0.4361
Step Size=8, Iteration 5, Accuracy: 0.4361


,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1008,D,DDD,1,1,-20.0
1,2,992,A,AAA,1,1,20.0
2,3,992,B,BBB,1,1,20.0
3,4,992,C,CCC,1,1,20.0


In [7]:
rankings = rating_system.calculate_ratings(
    step_sizes=[4,2,1,0.5],
    min_player_match_count=10
)
rankings

Step Size=4, Iteration 1, Accuracy: 0.43611
Step Size=4, Iteration 2, Accuracy: 0.43611
Step Size=4, Iteration 3, Accuracy: 0.43611
Step Size=2, Iteration 1, Accuracy: 0.43611
Step Size=1, Iteration 1, Accuracy: 0.43611
Step Size=0.5, Iteration 1, Accuracy: 0.43611
Step Size=0.5, Iteration 2, Accuracy: 0.43611


,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
